In [68]:
%matplotlib inline
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.metrics import mean_squared_error

In [69]:
#Join iata data
iata = pd.read_csv('data/IATA_US_CODES.csv')
iata['IATA Code']=iata['IATA Code'].astype(str)
iata['IATA Code']=iata['IATA Code'].str.rstrip()
iata=iata.set_index('IATA Code')
iata_dep=iata.add_suffix('_Dep')
iata_arr=iata.add_suffix('_Arr')
iata['2018']=iata['2018'].str.replace(',','')
iata['2017']=iata['2017'].str.replace(',','')
iata['2016']=iata['2016'].str.replace(',','')
iata['2015']=iata['2015'].str.replace(',','')
iata['2018']=iata['2018'].str.replace('\xa0','')
iata['2017']=iata['2017'].str.replace('\xa0','')
iata['2016']=iata['2016'].str.replace('\xa0','')
iata['2015']=iata['2015'].str.replace('\xa0','')
iata['State']=iata['State'].str.replace('\xa0','')
iata['Major city served']=iata['Major city served'].str.replace('New York/','')
iata['Major city served']=iata['Major city served'].str.replace('/St. Paul','')
iata['Major city served']=iata['Major city served'].str.replace('Baltimore/','')
iata['Major city served']=iata['Major city served'].str.replace('/Tacoma','')
iata['Major city served']=iata['Major city served'].str.replace('/Fort Worth','')
iata['Major city served']=iata['Major city served'].str.replace(', D.C.','')
iata

,Rank_2018,Airports (large hubs),Major city served,State,2018,2017,2016,2015
IATA Code,,,,,,,,
ATL,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732
LAX,2,Los Angeles International Airport,Los Angeles,CA,42626783,41232432,39636042,36351226
ORD,3,O'Hare International Airport,Chicago,IL,39874879,38593028,37589899,36305668
DFW,4,Dallas/Fort Worth International Airport,Dallas,TX,32800721,31816933,31283579,31589832
DEN,5,Denver International Airport,Denver,CO,31363573,29809097,28267394,26280043
JFK,6,John F. Kennedy International Airport,New York,NY,30541459,29533154,29239151,27782369
SFO,7,San Francisco International Airport,San Francisco,CA,27794154,26900048,25707101,24190549
SEA,8,Seattle–Tacoma International Airport,Seattle,WA,24894338,22639124,21887110,21231781
LAS,9,McCarran International Airport,Las Vegas,NV,23655285,23364393,22833267,21824231


In [70]:
data_weather = pd.read_csv('submissions/starting_kit_2/external_data.csv')
data_weather=data_weather.join(iata, on='AirPort', how='left')
data_weather['Major city served']=data_weather['Major city served'].str.replace('\xa0','')

data_weather['Major city served'].unique()

array(['Atlanta', 'Chicago', 'Los Angeles', 'Dallas', 'Denver',
       'New York', 'San Francisco', 'Charlotte', 'Las Vegas', 'Phoenix',
       'Houston', 'Miami', 'Orlando', 'Newark', 'Seattle', 'Minneapolis',
       'Detroit', 'Philadelphia', 'Boston'], dtype=object)

In [71]:
us_cities = pd.read_csv('data/uscities.csv')

In [72]:
data_weather=data_weather.merge(us_cities, left_on=['Major city served', 'State'],right_on=['city', 'state_id'], how='left')

In [73]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,Airports (large hubs),Major city served,State,2018,2017,2016,2015,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,Hartsfield–Jackson Atlanta International Airport,Atlanta,GA,51866464,50251964,50501858,49340732,Atlanta,Atlanta,GA,Georgia,13121,Fulton,13121|13089|13067,Fulton|DeKalb|Cobb,33.7627,-84.4225,5228750.0,1384.0,polygon,False,True,America/New_York,1,30334 30331 30332 30309 30308 30305 30307 3030...,1840016319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840059961
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840059961
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,LaGuardia Airport,New York,NY,15059648,14614802,14706123,14762593,New York,New York,NY,New York,36061,New York,36061,New York,40.6943,-73.9249,19354922.0,11083.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840059961
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,1

In [74]:
data_weather=data_weather.drop(['city_ascii','state_id','state_name','county_fips','county_name',
                                'county_fips_all','county_name_all','source','military',
                                'incorporated','timezone','zips','id','Major city served',
                               'Airports (large hubs)'], axis=1)

In [75]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1


In [76]:
from math import *

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [77]:
fuel = pd.read_csv('data/fuelprices.csv')
fuel=fuel.drop('Unnamed: 0',axis=1)
list(fuel['Date'].unique())

['2011-09-01',
 '2011-09-02',
 '2011-09-06',
 '2011-09-07',
 '2011-09-08',
 '2011-09-09',
 '2011-09-12',
 '2011-09-13',
 '2011-09-14',
 '2011-09-15',
 '2011-09-16',
 '2011-09-19',
 '2011-09-20',
 '2011-09-21',
 '2011-09-22',
 '2011-09-23',
 '2011-09-26',
 '2011-09-27',
 '2011-09-28',
 '2011-09-29',
 '2011-09-30',
 '2011-10-03',
 '2011-10-04',
 '2011-10-05',
 '2011-10-06',
 '2011-10-07',
 '2011-10-10',
 '2011-10-11',
 '2011-10-12',
 '2011-10-13',
 '2011-10-14',
 '2011-10-17',
 '2011-10-18',
 '2011-10-19',
 '2011-10-20',
 '2011-10-21',
 '2011-10-24',
 '2011-10-25',
 '2011-10-26',
 '2011-10-27',
 '2011-10-28',
 '2011-10-31',
 '2011-11-01',
 '2011-11-02',
 '2011-11-03',
 '2011-11-04',
 '2011-11-07',
 '2011-11-08',
 '2011-11-09',
 '2011-11-10',
 '2011-11-11',
 '2011-11-14',
 '2011-11-15',
 '2011-11-16',
 '2011-11-17',
 '2011-11-18',
 '2011-11-21',
 '2011-11-22',
 '2011-11-23',
 '2011-11-25',
 '2011-11-28',
 '2011-11-29',
 '2011-11-30',
 '2011-12-01',
 '2011-12-02',
 '2011-12-05',
 '2011-12-

In [78]:
data_weather=data_weather.merge(fuel, left_on='Date', right_on='Date', how='left')

In [79]:
data_weather[data_weather['U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)'].isna()]

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
9,2011-09-10,ATL,29,22,14,13,11,7,84,56,28,1019,1016,1015,16,16,16,21,9,32.0,0.00,0,NaN,321,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
10,2011-09-11,ATL,30,23,16,13,11,7,72,49,25,1020,1018,1017,16,16,16,21,6,37.0,0.00,2,NaN,294,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11024,2013-02-18,LGA,2,-2,-7,-12,-15,-17,51,39,26,1024,1022,1016,16,16,16,50,26,64.0,0.00,1,NaN,278,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11029,2013-02-23,LGA,4,3,2,2,1,-1,92,84,76,1025,1017,1009,16,7,0,34,18,42.0,6.86,8,Rain,53,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11030,2013-02-24,LGA,8,5,2,2,-1,-6,92,73,53,1014,1009,1007,16,14,5,45,22,56.0,0.76,7,Rain-Snow,327,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN


In [80]:
data_weather = data_weather.rename(
        columns={'U.S. Gulf Coast Kerosene-Type Jet Fuel Spot Price FOB (Dollars per Gallon)': 'Fuel_price'})

In [81]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,088"
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,NaN
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,005"


In [82]:
data_weather['Fuel_price'].fillna(method='ffill', inplace=True)

In [83]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,088"
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,"3,038"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,016"
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,"3,005"


In [84]:
data_weather[data_weather['Fuel_price'].isna()]

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price


In [85]:
data_weather['Fuel_price']=data_weather['Fuel_price'].str.replace(',','').astype(int)

In [86]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3088
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3005


In [65]:
#data_weather.to_csv('submissions/starting_kit/external_data.csv')

In [87]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3088
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3005


In [88]:
holiday = pd.read_csv('data/Jours_fériés_US.csv', sep=';', header=None)
holiday[0]=pd.to_datetime(holiday[0])
holiday

data_weather['Date']=pd.to_datetime(data_weather['Date'])


FileNotFoundError: [Errno 2] File b'data/Jours_f\xc3\xa9ri\xc3\xa9s_US.csv' does not exist: b'data/Jours_f\xc3\xa9ri\xc3\xa9s_US.csv'

In [89]:
data_weather = data_weather.merge(holiday, left_on='Date', right_on=0)
data_weather

NameError: name 'holiday' is not defined

In [90]:
data_weather=data_weather.drop(0, axis=1)

KeyError: '[0] not found in axis'

In [ ]:
data_weather=data_weather.rename(
        columns={1: 'Holiday'})

In [23]:
data_weather

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3088
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-01,LGA,7,5,3,-1,-3,-4,76,63,49,1008,1005,1002,16,16,16,34,21,42.0,0.00,6,NaN,320,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11036,2013-03-02,LGA,4,2,0,-2,-5,-6,82,65,48,1008,1007,1006,16,15,6,34,20,40.0,T,8,Snow,317,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11037,2013-03-03,LGA,4,2,-1,-5,-8,-9,69,55,40,1008,1006,1004,16,15,8,39,24,50.0,T,6,Snow,314,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3016
11038,2013-03-04,LGA,5,2,-2,-7,-8,-9,63,54,44,1012,1009,1008,16,16,16,47,31,60.0,0.00,3,NaN,313,21,NY,15059648,14614802,14706123,14762593,New York,40.6943,-73.9249,19354922.0,11083.0,1,3005


In [101]:
#data_weather.to_csv('submissions/starting_kit/external_data.csv')

In [91]:
data_weather = pd.read_csv('submissions/starting_kit/external_data.csv')
data_weather

,Unnamed: 0,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price,Holiday
0,0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3088,0
1,1,2011-09-01,ORD,34,28,22,23,22,19,84,70,56,1015,1013,1011,16,15,11,26,9,37.0,0.00,1,NaN,197,3,IL,39874879,38593028,37589899,36305668,Chicago,41.8373,-87.6862,8675982.0,4612.0,1,3088,0
2,2,2011-09-01,LAX,21,18,15,14,14,13,90,76,61,1012,1011,1009,16,11,4,23,13,NaN,0.00,5,Fog,265,2,CA,42626783,41232432,39636042,36351226,Los Angeles,34.1139,-118.4068,12815475.0,3295.0,1,3088,0
3,3,2011-09-01,DFW,38,33,27,17,14,11,51,35,19,1015,1013,1010,16,16,16,34,14,45.0,0.00,1,NaN,157,4,TX,32800721,31816933,31283579,31589832,Dallas,32.7936,-96.7662,5733259.0,1524.0,1,3088,0
4,4,2011-09-01,DEN,36,28,20,5,3,-2,34,22,9,1016,1007,1004,16,16,16,29,15,39.0,0.00,5,NaN,205,5,CO,31363573,29809097,28267394,26280043,Denver,39.7621,-104.8759,2787266.0,1774.0,1,3088,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,11035,2013-03-05,MSP,-1,-3,-6,-4,-5,-8,100,85,69,1027,1019,1015,16,6,1,27,15,40.0,10.41,8,Fog-Snow,44,17,MN,18363493,18409704,18123844,17634252,Minneapolis,44.9635,-93.2678,2926757.0,3019.0,1,3021,0
11036,11036,2013-03-05,DTW,2,-2,-6,-5,-6,-8,81,69,56,1021,1016,1010,16,16,16,32,16,42.0,0.00,5,NaN,58,19,MI,17437716,17036092,16826287,16255520,Detroit,42.3834,-83.1024,3522206.0,1873.0,1,3021,0
11037,11037,2013-03-05,PHL,9,4,-1,-2,-6,-7,70,53,36,1018,1016,1014,16,16,16,26,11,32.0,0.00,3,NaN,39,20,PA,15285948,14271243,14564419,15101318,Philadelphia,40.0077,-75.1339,5637884.0,4547.0,1,3021,0
11038,11038,2013-03-05,BOS,6,3,0,1,-3,-7,82,68,53,1018,1014,1007,16,16,14,39,17,52.0,T,7,Rain,359,16,MA,20010262,18759742,17749202,16290362,Boston,42.3188,-71.0846,4637537.0,5472.0,1,3021,0


In [92]:
data = data_weather.drop('Unnamed: 0', axis=1)
data

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price,Holiday
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3088,0
1,2011-09-01,ORD,34,28,22,23,22,19,84,70,56,1015,1013,1011,16,15,11,26,9,37.0,0.00,1,NaN,197,3,IL,39874879,38593028,37589899,36305668,Chicago,41.8373,-87.6862,8675982.0,4612.0,1,3088,0
2,2011-09-01,LAX,21,18,15,14,14,13,90,76,61,1012,1011,1009,16,11,4,23,13,NaN,0.00,5,Fog,265,2,CA,42626783,41232432,39636042,36351226,Los Angeles,34.1139,-118.4068,12815475.0,3295.0,1,3088,0
3,2011-09-01,DFW,38,33,27,17,14,11,51,35,19,1015,1013,1010,16,16,16,34,14,45.0,0.00,1,NaN,157,4,TX,32800721,31816933,31283579,31589832,Dallas,32.7936,-96.7662,5733259.0,1524.0,1,3088,0
4,2011-09-01,DEN,36,28,20,5,3,-2,34,22,9,1016,1007,1004,16,16,16,29,15,39.0,0.00,5,NaN,205,5,CO,31363573,29809097,28267394,26280043,Denver,39.7621,-104.8759,2787266.0,1774.0,1,3088,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-05,MSP,-1,-3,-6,-4,-5,-8,100,85,69,1027,1019,1015,16,6,1,27,15,40.0,10.41,8,Fog-Snow,44,17,MN,18363493,18409704,18123844,17634252,Minneapolis,44.9635,-93.2678,2926757.0,3019.0,1,3021,0
11036,2013-03-05,DTW,2,-2,-6,-5,-6,-8,81,69,56,1021,1016,1010,16,16,16,32,16,42.0,0.00,5,NaN,58,19,MI,17437716,17036092,16826287,16255520,Detroit,42.3834,-83.1024,3522206.0,1873.0,1,3021,0
11037,2013-03-05,PHL,9,4,-1,-2,-6,-7,70,53,36,1018,1016,1014,16,16,16,26,11,32.0,0.00,3,NaN,39,20,PA,15285948,14271243,14564419,15101318,Philadelphia,40.0077,-75.1339,5637884.0,4547.0,1,3021,0
11038,2013-03-05,BOS,6,3,0,1,-3,-7,82,68,53,1018,1014,1007,16,16,14,39,17,52.0,T,7,Rain,359,16,MA,20010262,18759742,17749202,16290362,Boston,42.3188,-71.0846,4637537.0,5472.0,1,3021,0


In [96]:
time = pd.read_csv('data/air_times.csv')
time = time.drop('Unnamed: 0', axis=1)
time

,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,AIR_TIME_MEAN
0,04A,"Lik, AK",OTZ,"Kotzebue, AK",192.0
1,04A,"Lik, AK",WTK,"Noatak, AK",46.0
2,05A,"Little Squaw, AK",FAI,"Fairbanks, AK",67.4
3,06A,"Kizhuyak, AK",ADQ,"Kodiak, AK",19.5
4,08A,"Elizabeth Island, AK",KEB,"Nanwalek, AK",8.0
...,...,...,...,...,...
24317,ZXN,"Clover Bay, AK",WMK,"Meyers Chuck, AK",29.0
24318,ZXN,"Clover Bay, AK",ZXM,"Saltery Cove, AK",11.5
24319,ZXO,"El Capitan, AK",WFB,"Ketchikan, AK",47.0
24320,ZXU,"North Kingstown, RI",ACK,"Nantucket, MA",38.0


In [100]:
data = data.merge(time, right_on='')

,Unnamed: 0,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price,Holiday
0,0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3088,0
1,1,2011-09-01,ORD,34,28,22,23,22,19,84,70,56,1015,1013,1011,16,15,11,26,9,37.0,0.00,1,NaN,197,3,IL,39874879,38593028,37589899,36305668,Chicago,41.8373,-87.6862,8675982.0,4612.0,1,3088,0
2,2,2011-09-01,LAX,21,18,15,14,14,13,90,76,61,1012,1011,1009,16,11,4,23,13,NaN,0.00,5,Fog,265,2,CA,42626783,41232432,39636042,36351226,Los Angeles,34.1139,-118.4068,12815475.0,3295.0,1,3088,0
3,3,2011-09-01,DFW,38,33,27,17,14,11,51,35,19,1015,1013,1010,16,16,16,34,14,45.0,0.00,1,NaN,157,4,TX,32800721,31816933,31283579,31589832,Dallas,32.7936,-96.7662,5733259.0,1524.0,1,3088,0
4,4,2011-09-01,DEN,36,28,20,5,3,-2,34,22,9,1016,1007,1004,16,16,16,29,15,39.0,0.00,5,NaN,205,5,CO,31363573,29809097,28267394,26280043,Denver,39.7621,-104.8759,2787266.0,1774.0,1,3088,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,11035,2013-03-05,MSP,-1,-3,-6,-4,-5,-8,100,85,69,1027,1019,1015,16,6,1,27,15,40.0,10.41,8,Fog-Snow,44,17,MN,18363493,18409704,18123844,17634252,Minneapolis,44.9635,-93.2678,2926757.0,3019.0,1,3021,0
11036,11036,2013-03-05,DTW,2,-2,-6,-5,-6,-8,81,69,56,1021,1016,1010,16,16,16,32,16,42.0,0.00,5,NaN,58,19,MI,17437716,17036092,16826287,16255520,Detroit,42.3834,-83.1024,3522206.0,1873.0,1,3021,0
11037,11037,2013-03-05,PHL,9,4,-1,-2,-6,-7,70,53,36,1018,1016,1014,16,16,16,26,11,32.0,0.00,3,NaN,39,20,PA,15285948,14271243,14564419,15101318,Philadelphia,40.0077,-75.1339,5637884.0,4547.0,1,3021,0
11038,11038,2013-03-05,BOS,6,3,0,1,-3,-7,82,68,53,1018,1014,1007,16,16,14,39,17,52.0,T,7,Rain,359,16,MA,20010262,18759742,17749202,16290362,Boston,42.3188,-71.0846,4637537.0,5472.0,1,3021,0


In [105]:
time['ORIGIN'] = time[time['ORIGIN'].apply(lambda x: x in data['AirPort']==True)]
time

,ORIGIN,ORIGIN_CITY_NAME,DEST,DEST_CITY_NAME,AIR_TIME_MEAN
0,NaN,"Lik, AK",OTZ,"Kotzebue, AK",192.0
1,NaN,"Lik, AK",WTK,"Noatak, AK",46.0
2,NaN,"Little Squaw, AK",FAI,"Fairbanks, AK",67.4
3,NaN,"Kizhuyak, AK",ADQ,"Kodiak, AK",19.5
4,NaN,"Elizabeth Island, AK",KEB,"Nanwalek, AK",8.0
...,...,...,...,...,...
24317,NaN,"Clover Bay, AK",WMK,"Meyers Chuck, AK",29.0
24318,NaN,"Clover Bay, AK",ZXM,"Saltery Cove, AK",11.5
24319,NaN,"El Capitan, AK",WFB,"Ketchikan, AK",47.0
24320,NaN,"North Kingstown, RI",ACK,"Nantucket, MA",38.0


In [113]:
vactions = pd.read_csv('data/fcal_United_States_2018.csv', sep=';', header=None)
vactions

,0,1,2
0,22.12.2017,02.01.2018,Christmas holidays
1,16.02.2018,19.02.2018,Sport holiday
2,26.03.2018,30.03.2018,Spring time holidays
3,16.06.2018,10.08.2018,Summer holidays
4,25.10.2018,26.10.2018,Autumn holidays
5,21.11.2018,23.11.2018,November Weekend
6,24.12.2018,01.01.2019,Christmas holidays
7,21.01.2019,23.01.2019,Bank holiday
8,18.02.2019,22.02.2019,Sport holiday
9,15.04.2019,19.04.2019,Spring time holidays


In [115]:
data.columns

Index(['Date', 'AirPort', 'Max TemperatureC', 'Mean TemperatureC',
       'Min TemperatureC', 'Dew PointC', 'MeanDew PointC', 'Min DewpointC',
       'Max Humidity', 'Mean Humidity', 'Min Humidity',
       'Max Sea Level PressurehPa', 'Mean Sea Level PressurehPa',
       'Min Sea Level PressurehPa', 'Max VisibilityKm', 'Mean VisibilityKm',
       'Min VisibilitykM', 'Max Wind SpeedKm/h', 'Mean Wind SpeedKm/h',
       'Max Gust SpeedKm/h', 'Precipitationmm', 'CloudCover', 'Events',
       'WindDirDegrees', 'Rank_2018', 'State', '2018', '2017', '2016', '2015',
       'city', 'lat', 'lng', 'population', 'density', 'ranking', 'Fuel_price',
       'Holiday'],
      dtype='object')

In [116]:
data

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees,Rank_2018,State,2018,2017,2016,2015,city,lat,lng,population,density,ranking,Fuel_price,Holiday
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129,1,GA,51866464,50251964,50501858,49340732,Atlanta,33.7627,-84.4225,5228750.0,1384.0,1,3088,0
1,2011-09-01,ORD,34,28,22,23,22,19,84,70,56,1015,1013,1011,16,15,11,26,9,37.0,0.00,1,NaN,197,3,IL,39874879,38593028,37589899,36305668,Chicago,41.8373,-87.6862,8675982.0,4612.0,1,3088,0
2,2011-09-01,LAX,21,18,15,14,14,13,90,76,61,1012,1011,1009,16,11,4,23,13,NaN,0.00,5,Fog,265,2,CA,42626783,41232432,39636042,36351226,Los Angeles,34.1139,-118.4068,12815475.0,3295.0,1,3088,0
3,2011-09-01,DFW,38,33,27,17,14,11,51,35,19,1015,1013,1010,16,16,16,34,14,45.0,0.00,1,NaN,157,4,TX,32800721,31816933,31283579,31589832,Dallas,32.7936,-96.7662,5733259.0,1524.0,1,3088,0
4,2011-09-01,DEN,36,28,20,5,3,-2,34,22,9,1016,1007,1004,16,16,16,29,15,39.0,0.00,5,NaN,205,5,CO,31363573,29809097,28267394,26280043,Denver,39.7621,-104.8759,2787266.0,1774.0,1,3088,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11035,2013-03-05,MSP,-1,-3,-6,-4,-5,-8,100,85,69,1027,1019,1015,16,6,1,27,15,40.0,10.41,8,Fog-Snow,44,17,MN,18363493,18409704,18123844,17634252,Minneapolis,44.9635,-93.2678,2926757.0,3019.0,1,3021,0
11036,2013-03-05,DTW,2,-2,-6,-5,-6,-8,81,69,56,1021,1016,1010,16,16,16,32,16,42.0,0.00,5,NaN,58,19,MI,17437716,17036092,16826287,16255520,Detroit,42.3834,-83.1024,3522206.0,1873.0,1,3021,0
11037,2013-03-05,PHL,9,4,-1,-2,-6,-7,70,53,36,1018,1016,1014,16,16,16,26,11,32.0,0.00,3,NaN,39,20,PA,15285948,14271243,14564419,15101318,Philadelphia,40.0077,-75.1339,5637884.0,4547.0,1,3021,0
11038,2013-03-05,BOS,6,3,0,1,-3,-7,82,68,53,1018,1014,1007,16,16,14,39,17,52.0,T,7,Rain,359,16,MA,20010262,18759742,17749202,16290362,Boston,42.3188,-71.0846,4637537.0,5472.0,1,3021,0
